###Mounting datalake gen 2 storage

In [2]:
tenantID = "1275dea2-b646-4021-8554-5a4100c6b5e0"

storageAccountName = "psdemostorageacc2"
fileSystemName = "psdemofs"

In [3]:

configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope = "key_vault_secrets", key = "azure-ad-app-id"), # retrieving app id from the key vault
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "key_vault_secrets", key = "azure-ad-secret-key"), # retrieving app secret from the key vault
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://{}@{}.dfs.core.windows.net/".format(fileSystemName, storageAccountName),
  mount_point = "/mnt/datalake",
  extra_configs = configs)


Out[4]: True

###Mounting blob storage

In [5]:
container_name = "fhvbases"
storage_account_name = "psdemoblob2"

In [6]:
dbutils.fs.mount(
  source = "wasbs://" + container_name + "@" + storage_account_name + ".blob.core.windows.net/",
  mount_point = "/mnt/storage",
  extra_configs = {
      "fs.azure.account.key." + storage_account_name + ".blob.core.windows.net" : dbutils.secrets.get(scope = "key_vault_secrets", key = "ps-demo-blob-access-key")
  }
)

Out[16]: True

###Making sure the mount is successfull by accessing the files in bith bolb storage and datalake gen 2 storage

In [8]:
dbutils.fs.ls('/mnt/storage/')

Out[1]: [FileInfo(path='dbfs:/mnt/storage/FhvBases.json', name='FhvBases.json', size=464836)]

In [9]:
dbutils.fs.head('/mnt/datalake/PaymentTypes.json')

Out[6]: '{"PaymentTypeID":1,"PaymentType":"Credit Card"}\r\n{"PaymentTypeID":2,"PaymentType":"Cash"}\r\n{"PaymentTypeID":3,"PaymentType":"No Charge"}\r\n{"PaymentTypeID":4,"PaymentType":"Dispute"}\r\n{"PaymentTypeID":5,"PaymentType":"Unknown"}\r\n{"PaymentTypeID":6,"PaymentType":"Voided Trip"}\r\n'

###Creating the database that will hold the trips data after the ETL process

In [11]:
%sql

create database if not exists taxi_service_warehouse;